In [5]:
from sklearn import preprocessing
import pandas as pd
from sklearn.linear_model import LinearRegression
import requests
import pandas as pd
import numpy as np
import indicators as ind
from backtester import Backtest
import pickle

In [9]:

PARAMS = {"backtest": "all"}
df = requests.get(
    "https://aipricepatterns.com/api/api/backtest", params=PARAMS)
data = df.json()['backtest']
data = pd.DataFrame(data)
data.rename({'close': 'Close'})
data = data.set_index('Time')
data.index = pd.to_datetime(data.index)
data['hours'] = data.index.hour
data['minutes'] = data.index.minute
data['dayofweek'] = data.index.dayofweek
print(data.head(1))

                              Close  Cumsum     High  Lag  Lag_short  \
Time                                                                   
2020-11-12 01:00:00+00:00  15609.36     0.0  15640.0    0          0   

                                Low     Open  Results Rule Rule_short  Signal  \
Time                                                                            
2020-11-12 01:00:00+00:00  15600.65  15635.5      0.0   no         no       0   

                           Signal_short  Volume  hours  minutes  dayofweek  
Time                                                                        
2020-11-12 01:00:00+00:00             0     0.0      1        0          3  


In [10]:
### Create value for momentum random forest strategy

In [11]:
class Strategies:

    @staticmethod
    def momentum(df, seq):
        signals = np.zeros(len(df))
        cons_day = 0
        for k in range(1, len(df)):
            price = df[k]
            prior_price = df[k-1]
            if price > prior_price:
                if cons_day < 0:
                    cons_day = 0
                cons_day += 1
            if price < prior_price:
                if cons_day > 0:
                    cons_day = 0
                cons_day -= 1
            if cons_day == seq:
                signals[k] = 1
            if cons_day == -seq:
                signals[k] = -1
        return signals


In [14]:
data_ml = data.copy()
data_ml = data_ml.rename(columns={'Close': 'close'})

In [17]:
data_ml['seq_1'] = Strategies.momentum(data_ml.close.values, seq=1)
data_ml['seq_2'] = Strategies.momentum(data_ml.close.values, seq=2)
data_ml['seq_3'] = Strategies.momentum(data_ml.close.values, seq=3)
data_ml['seq_4'] = Strategies.momentum(data_ml.close.values, seq=4)
data_ml['seq_5'] = Strategies.momentum(data_ml.close.values, seq=5)
data_ml['seq_6'] = Strategies.momentum(data_ml.close.values, seq=6)
data_ml['seq_7'] = Strategies.momentum(data_ml.close.values, seq=7)
data_ml['seq_8'] = Strategies.momentum(data_ml.close.values, seq=8)
data_ml['y'] = data_ml['close'].shift(-5).pct_change()
data_ml = data_ml.dropna()
X_rf = data_ml[['seq_1', 'seq_2', 'seq_3', 'seq_4', 'seq_5', 'seq_6', 'seq_7', 'seq_8', 'hours', 'minutes', 'dayofweek']] 
y_rf = data_ml['y']

In [ ]:
# Load random forestr model

In [19]:
threshold = -0.001
data_ml['signal_rf'] = np.where(data_ml['prdict_dt'] > threshold, 1, data_ml['prdict_dt'])
data_ml['signal_rf'] = np.where(data_ml['prdict_dt'] < threshold, -1, data_ml['signal'])

Time
2020-11-12 01:05:00+00:00    0.000487
2020-11-12 01:10:00+00:00   -0.000552
2020-11-12 01:15:00+00:00   -0.000629
2020-11-12 01:20:00+00:00    0.000859
2020-11-12 01:25:00+00:00    0.001497
                               ...   
2021-02-08 23:35:00+00:00    0.000000
2021-02-08 23:40:00+00:00    0.000000
2021-02-08 23:45:00+00:00    0.000000
2021-02-08 23:50:00+00:00    0.000000
2021-02-08 23:55:00+00:00    0.000000
Name: y, Length: 25619, dtype: float64

### Create values from decition tree mean reversing trading strategy

In [6]:
with open("rules_strategy.pickle", "rb") as p:
    data_rules = pickle.load(p)
data_rules = pd.DataFrame(data_rules)
data_rules['lag'] = data_rules['rules'].apply(lambda x: x.get('lag'))
data_rules['sigma'] = data_rules['rules'].apply(lambda x: x.get('sigma'))
data_rules['window_ma'] = data_rules['rules'].apply(lambda x: x.get('window_ma'))
data_rules['window_std'] = data_rules['rules'].apply(lambda x: x.get('window_std'))
X = data_rules[['lag', 'sigma', 'window_ma', 'window_std']]
y = data_rules['cumsum']